In [1]:
from agential.agents.critic.agent import Critic
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from agential.core.llm import LLM

from agential.core.fewshots.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT,
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.fever import (
    FEVER_FEWSHOT_EXAMPLES_COT,
    FEVER_FEWSHOT_EXAMPLES_DIRECT,
    FEVER_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.triviaqa import (
    TRIVIAQA_FEWSHOT_EXAMPLES_COT,
    TRIVIAQA_FEWSHOT_EXAMPLES_DIRECT,
    TRIVIAQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.ambignq import (
    AMBIGNQ_FEWSHOT_EXAMPLES_COT,
    AMBIGNQ_FEWSHOT_EXAMPLES_DIRECT,
    AMBIGNQ_FEWSHOT_EXAMPLES_REACT,
)
from agential.core.fewshots.gsm8k import (
    GSM8K_FEWSHOT_EXAMPLES_POT,
)
from agential.core.fewshots.svamp import (
    SVAMP_FEWSHOT_EXAMPLES_POT,
)
from agential.core.fewshots.tabmwp import (
    TABMWP_FEWSHOT_EXAMPLES_POT,
)
from agential.core.fewshots.humaneval import (
    HUMANEVAL_FEWSHOT_EXAMPLES_POT,
)
from agential.core.fewshots.mbpp import (
    MBPP_FEWSHOT_EXAMPLES_POT,
)

from agential.agents.critic.prompts import (
    # QA.
    CRITIC_INSTRUCTION_HOTPOTQA,
    CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,

    CRITIC_INSTRUCTION_FEVER,
    CRITIC_CRITIQUE_INSTRUCTION_FEVER,
    FEVER_FEWSHOT_EXAMPLES_CRITIC,

    CRITIC_INSTRUCTION_AMBIGNQ,
    CRITIC_CRITIQUE_INSTRUCTION_AMBIGNQ,
    AMBIGNQ_FEWSHOT_EXAMPLES_CRITIC,

    CRITIC_INSTRUCTION_TRIVIAQA,
    CRITIC_CRITIQUE_INSTRUCTION_TRIVIAQA,
    TRIVIAQA_FEWSHOT_EXAMPLES_CRITIC,

    # Math.
    CRITIC_POT_INSTRUCTION_GSM8K,
    CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_GSM8K,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,

    CRITIC_POT_INSTRUCTION_SVAMP,
    CRITIC_CRITIQUE_INSTRUCTION_SVAMP,
    SVAMP_FEWSHOT_EXAMPLES_CRITIC,
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_SVAMP,
    SVAMP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,

    CRITIC_POT_INSTRUCTION_TABMWP,
    CRITIC_CRITIQUE_INSTRUCTION_TABMWP,
    TABMWP_FEWSHOT_EXAMPLES_CRITIC,
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_TABMWP,
    TABMWP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,

    # Code.
    CRITIC_POT_INSTRUCTION_HUMANEVAL,
    CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC,
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,

    CRITIC_POT_INSTRUCTION_MBPP,
    CRITIC_CRITIQUE_INSTRUCTION_MBPP,
    MBPP_FEWSHOT_EXAMPLES_CRITIC,
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_MBPP,
    MBPP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
)

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings("ignore")

llm = LLM("gpt-3.5-turbo")
search = GoogleSearchAPIWrapper()

ValidationError: 1 validation error for GoogleSearchAPIWrapper
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass `google_api_key` as a named parameter. (type=value_error)

In [ ]:

import json

# Open and read the JSON file
with open('../data/hotpot_dev_v1_simplified.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data[:5])

In [ ]:
from agential.eval.metrics.classification import EM


agent = Critic(
    llm=llm, 
    benchmark="hotpotqa", 
    # kwargs.
    search=search, 
    evidence_length=400,
    num_results=8
)

samples = 20
num_correct = 0

for i in data[:samples]:

    question = i["question"]
    answer = i["answer"]
    use_tool = True

    out = agent.generate(
        question=question,
        examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,  # HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT, HOTPOTQA_FEWSHOT_EXAMPLES_REACT
        prompt=CRITIC_INSTRUCTION_HOTPOTQA,
        critique_examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
        critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
        additional_keys={},
        critique_additional_keys={},
        max_interactions=3,
        use_tool=use_tool,
        reset=True,
    )